In [ ]:
#Wikipedia History Log Extract - Hidden Treasures

In [ ]:
#Jupyter Notebook, Python3; to be improved. 

In [ ]:
#Copyright [Klaus Zinser] [name of copyright owner]
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#    http://www.apache.org/licenses/LICENSE-2.0
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

In [1]:
import requests
import sqlite3
import pandas as pd
import difflib
import random
import time

In [2]:
def get_revision_text(rev_id):
    """Takes full text of one Wikipedia Revision."""
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "revids": rev_id,
        "rvprop": "content"
    }
    response = requests.get(endpoint, params=params).json()
    page = next(iter(response['query']['pages'].values()))
    return page['revisions'][0]['*']

In [3]:
#Function definition
def calculate_diff(prev_text, curr_text):
    """Extracts the difference between two versions."""
    diff = difflib.unified_diff(prev_text.splitlines(keepends=True),
                                curr_text.splitlines(keepends=True),
                                fromfile='prev', tofile='curr', lineterm='')
    return '\n'.join(diff)

In [4]:
# Wikipedia-Article; for now British Family Court Judges; should work everwhere
article_title = "Judith_Parker"
#article_title = "James_Munby"
#article_title = "Ernest_Ryder"
#article_title = "Andrew_McFarlane_(judge)"

In [5]:
# Wikipedia API Endpoint
endpoint = f"https://en.wikipedia.org/w/api.php"

In [6]:
# SQL-Database Connection
conn = sqlite3.connect('wikipedia_history.db')
cursor = conn.cursor()

In [7]:
# Create Database Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    rev_id INTEGER PRIMARY KEY,
    parent_id INTEGER,
    user TEXT,
    timestamp TEXT,
    size INTEGER,
    comment TEXT
)
''')

In [8]:
#create database Table for difference text entries
cursor.execute('''
CREATE TABLE IF NOT EXISTS text_diffs (
    rev_id INTEGER,
    diff TEXT,
    FOREIGN KEY (rev_id) REFERENCES history(rev_id)
)
''')

In [9]:
# Parameter für die API-Anfrage
params = {
    "action": "query",
    "format": "json",
    "titles": article_title,
    "prop": "revisions",
    "rvlimit": "max",
    "rvprop": "ids|timestamp|user|size|comment"
}

In [10]:
# API-Anfrage
response = requests.get(endpoint, params=params).json()
pages = response['query']['pages']

In [ ]:
# print(response)

In [ ]:
# print(pages)

In [11]:
# Revisionsdaten extrahieren und in SQL einfügen
for page_id in pages:
    revisions = pages[page_id]['revisions']
    prev_rev = 1171811265
    for rev in revisions:
        cursor.execute('''
        INSERT OR IGNORE INTO history (rev_id, parent_id, user, timestamp, size, comment)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (rev['revid'], rev.get('parentid'), rev['user'], rev['timestamp'], rev['size'], rev.get('comment')))

        prev_text = get_revision_text(prev_rev) #version number from Line above
        curr_text = get_revision_text(rev['revid'])  # Current Revision-ID
        diff = calculate_diff(prev_text, curr_text)
        
        ## Example: Store Difference in Database #INSERT INTO text_diffs (prev_rev_id, rev_id, diff) #''', (prev_rev_id, curr_rev_id, diff))
        cursor.execute('''
        INSERT INTO text_diffs (rev_id, diff)
        VALUES (?, ?)
        ''', (rev['revid'], diff))
        prev_rev = rev['revid']
        #conn.commit()
        print(prev_rev)
        time.sleep(random.randint(3, 8))

1185160788
1173834512
1166851230
1166851138
1117733288
1109874560
1083093025
1070857400
1070857149
1066222690
1034367523
1032218741
888470284
882962230
882854716
882833399
882614812
858559695
856632022
849363019
834100609
813166849
784764585
763907194
763907061
760548983
726280599
726184829
726183945
726183837
726183658
726183417
726182835
726182672
726182024
726181987
726181919
726178843
726178742
726178211
726178181
726178105
726177982
726177727
726176995
726176502
726176256
725346948
725346769
724524678
714347259
714257320
694084651
613643354
613496484


In [12]:
# Änderungen speichern und Datenbankverbindung schließen #vor dem Commit existieren 2 Dateien, eine Journal. Danach eine
conn.commit()
conn.close()

In [ ]:
#Done under Apple OS
#Rename DB e.g. With Subject/Person Name
#Open DB with DB Browser for SQLite (etc)
#Export Table text_diffs to *.csv
#Open CSV File with Text Editor
#Print to pdf from TextEditor